In [8]:
# -*- coding: utf-8 -*-
# SWI Wedge Length Ratio – Smart Predictor
# Layout:
#   Left  = Inputs + large prediction readout
#   Right = Reference sketch (auto-fit)
#   Bottom = Buttons: Predict, Clear, Recall, Copy Result, Save Inputs, Load Inputs
# Tabs:
#   Predict, Batch, History, Article Info (CATENA, 2025; larger fonts)
# Deterministic predictions where possible, lazy model load, high-contrast dark mode, safe ttk theming.

import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from PIL import Image, ImageTk
import joblib
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime

# ------------------------------
# Config
# ------------------------------
MODEL_PATH = r"C:\Users\asus1\Desktop\CGB.joblib"   # model path (CatBoost/other)
IMAGE_PATH = r"C:/Users/asus1/Desktop/sketch22.png"  # shown on the right
APP_TITLE  = "SWI Wedge Length Ratio – Smart Predictor"
WINDOW_SIZE = "1200x780"

# Keep X1..X8 as internal keys for model compatibility; display scientific labels in UI.
FEATURE_KEYS = ['X1','X2','X3','X4','X5','X6','X7','X8']

# Order and labels EXACTLY as requested:
# X1 -> ρs/ρf , X2 -> K/Ko , X3 -> Qi/KoLo² , X4 -> i ,
# X5 -> Xi/Lo , X6 -> Yi/Lo , X7 -> Xb/Lo , X8 -> Db/Lo
LABELS = {
    'X1': "ρs/ρf   (Relative density)",
    'X2': "K/Ko    (Relative hydraulic conductivity)",
    'X3': "Qi/KoLo²  (Relative recharge rate)",
    'X4': "i       (Hydraulic gradient)",
    'X5': "Xi/Lo   (Relative well distance)",
    'X6': "Yi/Lo   (Relative well depth)",
    'X7': "Xb/Lo   (Relative barrier wall distance)",
    'X8': "Db/Lo   (Relative barrier wall depth)",
}

# Definitions shown as tooltips (English)
TOOLTIPS = {
    'X1': "ρs/ρf is the relative density (dimensionless).",
    'X2': "K/Ko is the relative hydraulic conductivity (dimensionless).",
    'X3': "Qi/KoLo² is the relative recharge rate (dimensionless).",
    'X4': "i is the hydraulic gradient (dimensionless).",
    'X5': "Xi/Lo is the relative well distance (dimensionless).",
    'X6': "Yi/Lo is the relative well depth (dimensionless).",
    'X7': "Xb/Lo is the relative barrier wall distance (dimensionless).",
    'X8': "Db/Lo is the relative barrier wall depth (dimensionless).",
}

# Keep only Baseline preset (plus placeholder)
PRESETS = {
    "— choose a preset —": None,
    "Baseline": [1.027, 1, 0.0006, 0.0121, 1, 0.313, 0.313, 0.138],
}

# High-contrast themes
THEME = {
    "light": {
        "bg":"#F7F9FC","fg":"#0D1B26","accent":"#2C7BE5","accent_fg":"#FFFFFF",
        "panel":"#FFFFFF","border":"#DDE6EF","muted":"#5C6B78","entry_bg":"#FFFFFF"
    },
    "dark": {
        "bg":"#0A0F14","fg":"#EAF2FA","accent":"#00D1FF","accent_fg":"#001018",
        "panel":"#111923","border":"#283545","muted":"#A8B2BD","entry_bg":"#0D141C"
    },
}

# ------------------------------
# Tooltips
# ------------------------------
class Tooltip:
    def __init__(self, widget, text, bg="#111", fg="#fff"):
        self.widget = widget; self.text = text; self.bg = bg; self.fg = fg; self.tip = None
        widget.bind("<Enter>", self.show); widget.bind("<Leave>", self.hide)
    def show(self, _=None):
        if self.tip or not self.text: return
        x = self.widget.winfo_rootx() + 30; y = self.widget.winfo_rooty() + 20
        self.tip = tw = tk.Toplevel(self.widget); tw.wm_overrideredirect(True); tw.wm_geometry(f"+{x}+{y}")
        lbl = tk.Label(tw, text=self.text, justify="left", background=self.bg, foreground=self.fg,
                       relief="solid", borderwidth=1, padx=8, pady=4, font=("Segoe UI", 9))
        lbl.pack()
    def hide(self, _=None):
        if self.tip: self.tip.destroy(); self.tip=None

# ------------------------------
# Deterministic model helpers (safe for CatBoost / XGB / sklearn)
# ------------------------------
def lock_model_deterministic(model_obj):
    """Try to make predictions deterministic and collect expected feature names."""
    expected_feature_names = None

    # XGBoost booster params (ignored by CatBoost/sklearn)
    try:
        booster = model_obj.get_booster()
        try:
            booster.set_param({"predictor": "cpu_predictor", "nthread": 1})
        except Exception:
            pass
    except Exception:
        pass

    # sklearn-style threading hints (ignored otherwise)
    try:
        model_obj.set_params(n_jobs=1, nthread=1)
    except Exception:
        pass

    # Try different sources for feature names
    try:
        if hasattr(model_obj, "feature_names_in_") and len(getattr(model_obj, "feature_names_in_")) > 0:
            expected_feature_names = list(model_obj.feature_names_in_)
    except Exception:
        pass
    if expected_feature_names is None:
        try:
            feats = getattr(model_obj, "feature_names_", None)
            if feats and len(feats) > 0:
                expected_feature_names = list(feats)
        except Exception:
            pass
    if expected_feature_names is None:
        try:
            expected_feature_names = list(model_obj.get_booster().feature_names)
        except Exception:
            pass

    # Final fallback: use UI keys to avoid false mismatch warnings
    if not expected_feature_names:
        expected_feature_names = FEATURE_KEYS[:]
    return expected_feature_names

def df_ordered_row(values_dict, expected_feature_names):
    ordered_cols = list(map(str, expected_feature_names))
    row = [values_dict[c] for c in ordered_cols]
    X = pd.DataFrame([row], columns=ordered_cols).astype(np.float32)
    return X

# ------------------------------
# Paper metadata (superscripts kept)
# ------------------------------
PAPER_TITLE  = "Simulating the Effectiveness of Artificial Recharge and Cutoff Walls for Saltwater Intrusion Control with Explainable ML and GUI Deployment"
AUTHORS_LINE = "Developers: Mohamed Kamel Elshaarawy (https://orcid.org/0000-0002-1793-5617)  &  Asaad M. Armanuos (https://orcid.org/0000-0002-4134-0429) "
AFFIL_1      = "¹ Civil Engineering Department, Faculty of Engineering, Horus University-Egypt, New Damietta 34517, Egypt; melshaarawy@horus.edu.eg (M.K.E.)"
AFFIL_2      = "² Irrigation and Hydraulics Engineering Department, Faculty of Engineering, Tanta University, Tanta 31733, Egypt; asaad.matter@f-eng.tanta.edu.eg (A.M.A.)"
CORR         = "*Corresponding author"
JOURNAL      = "CATENA"
YEAR         = "2025"

APA_CITATION = (
    "Elshaarawy, M. K., & Armanuos, A. M. (2025). "
    "Simulating the effectiveness of artificial recharge and cutoff walls for saltwater intrusion control "
    "with explainable ML and GUI deployment. CATENA."
)

# ------------------------------
# App shell
# ------------------------------
root = tk.Tk()
root.title(APP_TITLE)
root.geometry(WINDOW_SIZE)
root.minsize(1080, 680)

style = ttk.Style(); style.theme_use("clam")
current_theme = "light"

# ------------------------------
# Header
# ------------------------------
header = tk.Frame(root); header.pack(fill="x", padx=16, pady=(16, 8))
title_label = ttk.Label(header, text=PAPER_TITLE, font=("Segoe UI", 16, "bold"), wraplength=1120, justify="left")
subtitle_label = ttk.Label(header, text=AUTHORS_LINE, font=("Segoe UI", 12))
title_label.pack(anchor="w"); subtitle_label.pack(anchor="w")
title_badge = tk.Label(header, text="LIVE", font=("Segoe UI", 11, "bold"), padx=10, pady=3)
title_badge.place(relx=1.0, x=-10, y=5, anchor="ne")

# ------------------------------
# Notebook
# ------------------------------
notebook = ttk.Notebook(root); notebook.pack(fill="both", expand=True, padx=16, pady=(0, 8))
tab_predict = ttk.Frame(notebook); tab_batch = ttk.Frame(notebook); tab_history = ttk.Frame(notebook); tab_article = ttk.Frame(notebook)
notebook.add(tab_predict, text="Predict"); notebook.add(tab_batch, text="Batch")
notebook.add(tab_history, text="History"); notebook.add(tab_article, text="Article Info")

# ------------------------------
# Predict tab: LEFT (inputs/result) | RIGHT (image)
# ------------------------------
predict_grid = ttk.Frame(tab_predict); predict_grid.pack(fill="both", expand=True, padx=8, pady=(8,0))
predict_grid.grid_columnconfigure(0, weight=3)
predict_grid.grid_columnconfigure(1, weight=2)
predict_grid.grid_rowconfigure(0, weight=1)

# Left panel
left_panel = ttk.Frame(predict_grid); left_panel.grid(row=0, column=0, sticky="nsew", padx=(0,8))
left_panel.grid_columnconfigure(0, weight=1)

# Big prediction readout
pred_card = ttk.LabelFrame(left_panel, text="Prediction", padding=12)
pred_card.grid(row=0, column=0, sticky="ew", pady=(0,8))
ttk.Label(pred_card, text="Predicted Relative SWI wedge length (L/Lo):", font=("Segoe UI", 11, "bold")).pack(anchor="w")
result_value = ttk.Label(pred_card, text="—", font=("Segoe UI", 30, "bold"))
result_value.pack(anchor="w", pady=(6,0))

# Inputs card
inputs_card = ttk.LabelFrame(left_panel, text="Input Parameters (Dimensionless)", padding=12)
inputs_card.grid(row=1, column=0, sticky="nsew")
inputs_card.grid_columnconfigure(1, weight=1)

entry_vars = {}; last_inputs = None; expected_feature_names = FEATURE_KEYS[:]  # default to UI keys
model = None

def make_row(parent, key, label_text, r):
    lbl = ttk.Label(parent, text=label_text, font=("Segoe UI", 11))
    lbl.grid(row=r, column=0, sticky="w", padx=(2,8), pady=6)
    sv = tk.StringVar()
    e = ttk.Entry(parent, textvariable=sv, font=("Segoe UI", 12), width=18, justify="center")
    e.grid(row=r, column=1, sticky="ew", padx=(0,4), pady=6)
    sv.widget = e
    entry_vars[key] = sv
    Tooltip(lbl, TOOLTIPS.get(key, ""), bg="#222", fg="#fff")

for i, k in enumerate(FEATURE_KEYS): make_row(inputs_card, k, LABELS[k], i)

# Presets (only Baseline)
preset_frame = ttk.Frame(inputs_card); preset_frame.grid(row=len(FEATURE_KEYS), column=0, columnspan=2, sticky="ew", pady=(10,0))
ttk.Label(preset_frame, text="Preset:").pack(side="left", padx=(0,8))
preset_combo = ttk.Combobox(preset_frame, values=list(PRESETS.keys()), state="readonly", width=26); preset_combo.pack(side="left")
def apply_preset_from_combo(_=None):
    name = preset_combo.get()
    if not name: return
    vals = PRESETS[name]
    if vals is None:  # ignore placeholder
        return
    for k, v in zip(FEATURE_KEYS, vals): entry_vars[k].set(str(v))
preset_combo.bind("<<ComboboxSelected>>", apply_preset_from_combo)

# Right panel (image)
right_panel = ttk.LabelFrame(predict_grid, text="Reference Sketch", padding=8)
right_panel.grid(row=0, column=1, sticky="nsew")
right_panel.grid_rowconfigure(0, weight=1); right_panel.grid_columnconfigure(0, weight=1)
image_label = tk.Label(right_panel, bd=0)  # classic tk for bg theming
image_label.grid(row=0, column=0, sticky="nsew")
image_obj = None; photo_obj = None

def load_right_image(path):
    global image_obj, photo_obj
    if not path or not os.path.exists(path):
        image_label.config(text="No image found", font=("Segoe UI", 12)); return
    try:
        image_obj = Image.open(path).convert("RGBA"); _resize_right_image()
    except Exception as e:
        image_label.config(text=f"Image error:\n{e}", font=("Segoe UI", 11))

def _resize_right_image(event=None):
    global photo_obj
    if image_obj is None: return
    card_w = max(right_panel.winfo_width() - 16, 50)
    card_h = max(right_panel.winfo_height() - 16, 50)
    if card_w < 50 or card_h < 50: return
    img_w, img_h = image_obj.size
    scale = min(card_w / img_w, card_h / img_h)
    new_size = (max(1, int(img_w * scale)), max(1, int(img_h * scale)))
    img_resized = image_obj.resize(new_size, Image.LANCZOS)
    photo_obj = ImageTk.PhotoImage(img_resized)
    image_label.config(image=photo_obj, text="")
right_panel.bind("<Configure>", _resize_right_image)

# ------------------------------
# Bottom button bar (ALL buttons)
# ------------------------------
bottom_bar = tk.Frame(root, height=54)
bottom_bar.pack(fill="x", padx=16, pady=(0, 12))

predict_btn = ttk.Button(bottom_bar, text="Predict", style="Accent.TButton")
clear_btn   = ttk.Button(bottom_bar, text="Clear")
recall_btn  = ttk.Button(bottom_bar, text="Recall Last", state=tk.DISABLED)
copy_btn    = ttk.Button(bottom_bar, text="Copy Result")
save_btn    = ttk.Button(bottom_bar, text="Save Inputs")
load_btn    = ttk.Button(bottom_bar, text="Load Inputs")

for b in (predict_btn, clear_btn, recall_btn, copy_btn, save_btn, load_btn):
    b.pack(side="left", padx=6)

# ------------------------------
# Batch tab
# ------------------------------
batch_card = ttk.LabelFrame(tab_batch, text="Batch Predictions (CSV → CSV)", padding=12)
batch_card.pack(fill="both", expand=True, padx=8, pady=8)
ttk.Label(batch_card, text="Input CSV with columns: X1..X8").grid(row=0, column=0, sticky="w", pady=(0,6))
batch_path_var = tk.StringVar()
ttk.Entry(batch_card, textvariable=batch_path_var).grid(row=1, column=0, sticky="ew")
def browse_csv():
    p = filedialog.askopenfilename(filetypes=[("CSV files","*.csv")])
    if p: batch_path_var.set(p)
ttk.Button(batch_card, text="Browse…", command=browse_csv).grid(row=1, column=1, padx=6)
run_batch_btn = ttk.Button(batch_card, text="Run Batch"); run_batch_btn.grid(row=1, column=2, padx=6)
batch_card.columnconfigure(0, weight=1)

# ------------------------------
# History tab
# ------------------------------
history_card = ttk.LabelFrame(tab_history, text="Session History", padding=8)
history_card.pack(fill="both", expand=True, padx=8, pady=8)
cols = ["Time"] + FEATURE_KEYS + ["Prediction"]
tree = ttk.Treeview(history_card, columns=cols, show="headings", height=18)
for c in cols: tree.heading(c, text=c); tree.column(c, width=110, stretch=True)
tree.pack(fill="both", expand=True)
def push_history(vals_dict, pred_val):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row = [timestamp] + [vals_dict[k] for k in FEATURE_KEYS] + [round(float(pred_val), 6)]
    tree.insert("", "end", values=row)
def export_history():
    items = tree.get_children()
    if not items: messagebox.showinfo("History", "Nothing to export yet."); return
    rows = [tree.item(i, "values") for i in items]
    out_df = pd.DataFrame(rows, columns=cols)
    p = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV","*.csv")])
    if not p: return
    out_df.to_csv(p, index=False)
ttk.Button(history_card, text="Export History CSV", command=export_history).pack(anchor="e", pady=(6,0))

# ------------------------------
# Article Info tab (CATENA, 2025)
# ------------------------------
article_card = ttk.LabelFrame(tab_article, text="Article & Authors", padding=14)
article_card.pack(fill="both", expand=True, padx=8, pady=8)

ttk.Label(article_card, text=PAPER_TITLE, font=("Segoe UI", 20, "bold"), wraplength=1120, justify="left").pack(anchor="w", pady=(4,12))
ttk.Label(article_card, text=AUTHORS_LINE, font=("Segoe UI", 16, "bold"), wraplength=1120, justify="left").pack(anchor="w", pady=(0,10))
ttk.Label(article_card, text=AFFIL_1, font=("Segoe UI", 14), wraplength=1120, justify="left").pack(anchor="w")
ttk.Label(article_card, text=AFFIL_2, font=("Segoe UI", 14), wraplength=1120, justify="left").pack(anchor="w")
ttk.Label(article_card, text=CORR,   font=("Segoe UI", 14), wraplength=1120, justify="left").pack(anchor="w", pady=(6,10))
ttk.Label(article_card, text=f"{JOURNAL} ({YEAR})", font=("Segoe UI", 14, "italic")).pack(anchor="w", pady=(0,10))

def copy_citation():
    root.clipboard_clear(); root.clipboard_append(APA_CITATION)
    status("Citation copied")
ttk.Button(article_card, text="Copy APA Citation", command=copy_citation).pack(anchor="w", pady=(6,0))

# ------------------------------
# Actions
# ------------------------------
def status(msg):  # simple status in title bar
    root.title(f"{APP_TITLE} — {msg}")

def all_valid_floats():
    try:
        for k in FEATURE_KEYS: float(entry_vars[k].get().strip())
        return True
    except Exception:
        return False

def update_recall_state():
    recall_btn.config(state=(tk.NORMAL if last_inputs is not None else tk.DISABLED))

def collect_values_as_dataframe():
    vals = {k: float(entry_vars[k].get().strip()) for k in FEATURE_KEYS}
    X = df_ordered_row(vals, expected_feature_names); return X, vals

def do_predict(_=None):
    global last_inputs
    if model is None:
        messagebox.showerror("Model", "Model is not loaded."); return
    if not all_valid_floats():
        messagebox.showerror("Input", "Please enter valid numeric values for all eight inputs."); return
    try:
        # Hint for XGBoost if applicable (safe no-op for CatBoost/sklearn)
        try:
            model.get_booster().set_param({"predictor": "cpu_predictor", "nthread": 1})
        except Exception:
            pass
        X, vals = collect_values_as_dataframe()
        pred = model.predict(X)
        pred_val = float(np.array(pred).ravel()[0])
        result_value.config(text=f"{pred_val:.6f}")
        status(f"Predicted {pred_val:.6f}")
        push_history(vals, pred_val)
        last_inputs = [vals[k] for k in FEATURE_KEYS]; update_recall_state()
    except Exception as e:
        messagebox.showerror("Prediction Error", str(e))

def do_clear():
    result_value.config(text="—")
    for k in FEATURE_KEYS: entry_vars[k].set("")
    status("Cleared inputs.")

def do_recall():
    if last_inputs is None:
        messagebox.showinfo("Recall", "No previous inputs yet."); return
    for k, v in zip(FEATURE_KEYS, last_inputs): entry_vars[k].set(str(v))
    status("Recalled last inputs.")

def copy_result():
    txt = result_value.cget("text")
    if not txt or txt == "—":
        messagebox.showinfo("Copy", "No prediction to copy yet."); return
    root.clipboard_clear(); root.clipboard_append(txt)
    status("Prediction copied")

def save_inputs():
    data = {k: entry_vars[k].get().strip() for k in FEATURE_KEYS}
    path = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON","*.json")])
    if not path: return
    with open(path, "w") as f: json.dump(data, f, indent=2)
    status(f"Inputs saved: {os.path.basename(path)}")

def load_inputs():
    path = filedialog.askopenfilename(filetypes=[("JSON","*.json")])
    if not path: return
    with open(path, "r") as f: data = json.load(f)
    for k in FEATURE_KEYS:
        if k in data: entry_vars[k].set(str(data[k]))
    status(f"Inputs loaded: {os.path.basename(path)}")

predict_btn.config(command=do_predict)
clear_btn.config(command=do_clear)
recall_btn.config(command=do_recall)
copy_btn.config(command=copy_result)
save_btn.config(command=save_inputs)
load_btn.config(command=load_inputs)
root.bind("<Return>", do_predict)

def run_batch():
    if model is None:
        messagebox.showerror("Model", "Model is not loaded."); return
    in_path = batch_path_var.get().strip()
    if not in_path or not os.path.exists(in_path):
        messagebox.showerror("Batch", "Please choose a valid CSV file."); return
    try:
        df = pd.read_csv(in_path)
        missing = [c for c in FEATURE_KEYS if c not in df.columns]
        if missing: raise ValueError(f"CSV missing columns: {missing}")
        X_cols = df_ordered_row({k: 0.0 for k in FEATURE_KEYS}, expected_feature_names).columns
        vals = df[X_cols].astype(np.float32)
        try:
            model.get_booster().set_param({"predictor": "cpu_predictor", "nthread": 1})
        except Exception:
            pass
        preds = model.predict(vals)
        out_df = df.copy(); out_df["Pred_L_over_Lo"] = np.array(preds).ravel()
        base, ext = os.path.splitext(in_path); out_path = f"{base}_with_predictions.csv"
        out_df.to_csv(out_path, index=False)
        messagebox.showinfo("Batch", f"Saved predictions:\n{out_path}")
        status("Batch done")
    except Exception as e:
        messagebox.showerror("Batch Error", str(e))
run_batch_btn.config(command=run_batch)

# ------------------------------
# High-contrast theming
# ------------------------------
def apply_theme():
    colors = THEME[current_theme]
    # classic tk backgrounds
    root.configure(bg=colors["bg"]); header.configure(bg=colors["bg"])
    title_badge.configure(bg=colors["accent"], fg=colors["accent_fg"])
    image_label.configure(bg=colors["panel"])
    # ttk base
    style.configure("TFrame", background=colors["bg"])
    style.configure("TLabel", background=colors["bg"], foreground=colors["fg"])
    style.configure("TLabelframe", background=colors["panel"], foreground=colors["fg"], bordercolor=colors["border"])
    style.configure("TLabelframe.Label", background=colors["panel"], foreground=colors["fg"])
    style.configure("TEntry", fieldbackground=colors["entry_bg"], foreground=colors["fg"])
    style.map("TEntry",
              fieldbackground=[('disabled', colors["panel"])],
              foreground=[('disabled', colors["muted"])])
    style.configure("Treeview", background=colors["panel"], fieldbackground=colors["panel"], foreground=colors["fg"])
    style.configure("Treeview.Heading", background=colors["panel"], foreground=colors["fg"])
    # Buttons (accent + regular)
    style.configure("Accent.TButton", font=("Segoe UI", 11, "bold"))
    style.map("Accent.TButton",
              foreground=[('!disabled', colors["accent_fg"]), ('disabled', colors["muted"])],
              background=[('!disabled', colors["accent"]), ('active', colors["accent"]), ('pressed', colors["accent"])])
    style.configure("TButton", foreground=colors["fg"])
    style.map("TButton", foreground=[('disabled', colors["muted"])])

def toggle_dark_mode():
    global current_theme
    current_theme = "dark" if current_theme == "light" else "light"
    apply_theme()
    status(f"Theme: {current_theme.title()}")

# ------------------------------
# Menu
# ------------------------------
menubar = tk.Menu(root); root.config(menu=menubar)
file_menu = tk.Menu(menubar, tearoff=0); view_menu = tk.Menu(menubar, tearoff=0); help_menu = tk.Menu(menubar, tearoff=0)
menubar.add_cascade(label="File", menu=file_menu); menubar.add_cascade(label="View", menu=view_menu); menubar.add_cascade(label="Help", menu=help_menu)
view_menu.add_command(label="Toggle High-Contrast Dark Mode  (Ctrl+D)", command=toggle_dark_mode)
root.bind("<Control-d>", lambda e: toggle_dark_mode())
help_menu.add_command(label="About", command=lambda: messagebox.showinfo("About",
    "SWI Wedge Length Ratio – Smart Predictor\n"
    "Left = inputs, Right = sketch, Bottom = full button set.\n"
    "Article Info tab shows CATENA (2025) metadata."))

# ------------------------------
# Model load (lazy) + initial image/theme
# ------------------------------
def load_model_initial():
    global model, expected_feature_names
    if not os.path.exists(MODEL_PATH):
        status("Model not found"); return
    try:
        status("Loading model…")
        m = joblib.load(MODEL_PATH)
        expected_feature_names = lock_model_deterministic(m)
        model = m
        status("Model loaded")
        # Show feature-count warning only if n_features_in_ > 0 (avoid the "0 features" false warning)
        try:
            nfi = getattr(model, "n_features_in_", 0)
            if isinstance(nfi, (int, np.integer)) and (nfi > 0) and (nfi != len(FEATURE_KEYS)):
                messagebox.showwarning(
                    "Model Feature Mismatch",
                    f"Model expects {nfi} features, but UI is set up for {len(FEATURE_KEYS)}."
                )
        except Exception:
            pass
    except Exception as e:
        messagebox.showerror("Model Error", f"Couldn't load default model:\n{e}")
        status("Model load failed")

apply_theme()
load_right_image(IMAGE_PATH)
status("Ready")
root.bind("<Configure>", lambda e: _resize_right_image())  # keep image fitted
root.after(100, load_model_initial)  # lazy load so UI appears immediately
root.mainloop()
